In [1]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [2]:
pd.options.display.max_columns=None

In [3]:
df_esios=pd.read_csv('../data/datos_esios.csv', sep=';')

In [4]:
df_esios.head()

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2
0,2015-10-01 00:00:00,2196.1,24517.1,8203.3,5355.4,4705.3,56.65,38.56,56.31,58.30
1,2015-10-01 01:00:00,2064.4,23168.6,8286.8,5355.4,4705.3,53.50,31.27,54.25,56.60
2,2015-10-01 02:00:00,1866.5,22437.2,8301.3,5355.4,4705.3,49.69,30.66,49.69,51.58
3,2015-10-01 03:00:00,1660.9,22281.3,8281.3,5355.4,4705.3,49.10,29.14,49.10,49.57
4,2015-10-01 04:00:00,1373.9,21909.7,8247.7,5355.4,4705.3,51.25,29.40,51.25,52.00


In [5]:
df_esios.describe()

,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2
count,26304.000000,26304.000000,26304.000000,26304.000000,26304.000000,26304.000000,26304.000000,26304.00000,26304.000000
mean,5479.286014,28672.990169,4409.936025,4420.339625,4188.581794,48.739281,42.105033,48.46574,48.474287
std,3185.927399,4616.496856,2216.248299,474.038209,469.363759,14.954689,20.799903,15.09524,15.110244
min,240.800000,17728.400000,546.000000,2461.500000,2315.100000,2.060000,-31.820000,0.00000,0.000000
25%,3018.400000,24734.150000,2316.975000,4125.800000,3797.600000,40.137500,29.740000,40.05000,40.100000
50%,4822.950000,28920.600000,4634.000000,4483.200000,4148.100000,49.550000,39.165000,49.00000,48.950000
75%,7289.250000,32267.650000,6282.750000,4799.700000,4705.300000,59.450000,51.562500,58.90000,58.710000
max,16952.600000,41297.900000,8768.300000,9481.400000,8270.600000,101.990000,874.010000,102.49000,103.610000


In [6]:
df_esios['datetime'].describe()  #tenemos repetidos los datos para el dia de 25 horas.

count                   26304
unique                  26301
top       2017-10-29 02:00:00
freq                        2
Name: datetime, dtype: object

#### 1) Columna a predecir:

"reservar"

Nuestra clasificación será:

* Si PMI1 > PMD -> 0

* Si PMI1 < PMD -> 1

* Si PMI1 == PMD:

    * Si PMI2 > PMD -> 0
    * Si PMI2 <= PMD -> 1

En la descarga y exploración de datos esios se obtuvo que de los 26.304 datos, hay 3.493 casos en los que el precio del mercado diario coincidió con el del intra, y de esos, 793 horas en las que además el intradiario 2 tampoco subió/bajó con respecto al diario.

In [7]:
df_esios_precio=df_esios[['PRECIO_MD_ESP','PRECIO_I1','PRECIO_I2','PRECIO_MD_FR']]

In [8]:
reserva=np.zeros(len(df_esios_precio['PRECIO_MD_ESP']))
for k,v in df_esios_precio.iterrows():
    a=v['PRECIO_MD_ESP']
    b=v['PRECIO_I1']
    c=v['PRECIO_I2']
    if (a<b) | ((a==b) & (c>b)):
        reserva[k]=0
    else:
        reserva[k]=1  
df_esios['RESERVA']=reserva

In [9]:
df_esios.sample(10)  #comprobamos aleatoriamente que ok

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA
6016,2016-06-07 16:00:00,2914.2,31982.2,5383.4,3951.2,4135.3,45.75,30.18,44.75,38.00,1.0
25982,2018-09-17 14:00:00,4163.2,34492.2,6080.7,4182.3,4705.3,71.68,64.32,68.69,70.64,1.0
4126,2016-03-20 21:00:00,3004.1,30951.1,4298.1,4320.0,4148.1,45.98,28.00,43.00,40.63,1.0
19315,2017-12-13 18:00:00,10068.7,38119.5,7886.7,4443.5,4353.2,77.74,101.70,79.24,79.23,0.0
18632,2017-11-15 07:00:00,3141.6,32073.9,8015.4,4463.6,4355.1,67.07,86.97,69.07,69.17,0.0
7313,2016-07-31 17:00:00,6313.0,27371.0,2608.8,4293.6,4135.3,33.99,25.25,34.97,33.99,0.0
5095,2016-04-30 07:00:00,6236.3,21761.2,863.5,3108.9,4354.3,25.45,25.45,21.63,24.45,1.0
24436,2018-07-15 04:00:00,2201.2,21675.0,3621.9,4174.6,4705.3,58.00,42.29,56.00,53.98,1.0
14163,2017-05-13 03:00:00,7051.6,22186.2,2858.3,4829.4,4268.2,38.00,28.52,39.00,46.00,0.0
4837,2016-04-19 13:00:00,8889.5,32824.1,1394.0,2651.5,4354.3,23.00,23.00,22.00,23.00,1.0


#### 2) Interconexión

Creamos columna ACOPLADO_FR:

* Si Precio FR MD == Precio España MD están acoplados (1) si no no (0)*

In [10]:
acoplado=np.zeros(len(df_esios_precio['PRECIO_MD_ESP']))
for k,v in df_esios_precio.iterrows():
    a=v['PRECIO_MD_ESP']
    b=v['PRECIO_MD_FR']
    if (a==b):
        acoplado[k]=1
    else:
        acoplado[k]=0  
df_esios['ACOPLADO_FR']=acoplado

In [11]:
df_esios.sample(10) #comprobamos aleatoriamente que ok

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR
14467,2017-05-25 19:00:00,2874.3,31873.4,7138.4,4529.7,4050.6,53.02,33.92,54.00,53.02,0.0,0.0
16856,2017-09-02 08:00:00,6177.7,23510.7,1831.0,4799.7,4135.3,37.09,35.10,34.87,34.88,1.0,0.0
25140,2018-08-13 12:00:00,1237.8,33781.0,4463.4,4206.4,4705.3,67.30,61.91,65.90,66.80,1.0,0.0
22163,2018-04-11 11:00:00,13041.7,35825.1,2366.0,4444.6,4354.5,49.92,51.63,49.92,47.90,1.0,0.0
2791,2016-01-25 06:00:00,6202.2,25713.8,3433.8,4529.7,4705.3,33.34,33.34,29.94,30.99,1.0,1.0
4753,2016-04-16 01:00:00,9106.3,24259.4,1027.5,2997.3,3797.1,19.33,19.70,19.24,22.23,1.0,0.0
15091,2017-06-20 19:00:00,3612.9,36615.9,7710.1,4150.2,4705.3,56.00,39.71,58.50,58.00,0.0,0.0
9855,2016-11-14 14:00:00,8852.0,31592.4,5875.7,4293.6,3573.3,57.57,111.16,56.52,56.26,1.0,0.0
18540,2017-11-11 11:00:00,6244.1,29703.5,5999.8,4116.2,3433.7,57.97,57.97,59.27,61.08,0.0,1.0
4156,2016-03-22 03:00:00,5911.7,22573.0,2883.6,3972.3,4148.1,26.94,26.94,24.30,25.94,1.0,1.0


#### 3) MW Carbón disponible

Sumamos la potencia disponible de carbón. Nos interesa el total de MW disponibles.

In [12]:
df_esios['POT_DISP_CARBON']=[x['POT_DISP_HULLA_ANT'] + x['POT_DISP_HULA_SB'] for _,x in df_esios.iterrows()]

In [13]:
df_esios.head()

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR,POT_DISP_CARBON
0,2015-10-01 00:00:00,2196.1,24517.1,8203.3,5355.4,4705.3,56.65,38.56,56.31,58.30,1.0,0.0,10060.7
1,2015-10-01 01:00:00,2064.4,23168.6,8286.8,5355.4,4705.3,53.50,31.27,54.25,56.60,0.0,0.0,10060.7
2,2015-10-01 02:00:00,1866.5,22437.2,8301.3,5355.4,4705.3,49.69,30.66,49.69,51.58,0.0,0.0,10060.7
3,2015-10-01 03:00:00,1660.9,22281.3,8281.3,5355.4,4705.3,49.10,29.14,49.10,49.57,0.0,0.0,10060.7
4,2015-10-01 04:00:00,1373.9,21909.7,8247.7,5355.4,4705.3,51.25,29.40,51.25,52.00,0.0,0.0,10060.7


#### 4) Datos pasados

Necesitamos el carbón casado el día anterior (ya que el carbón que se va a casar ese día no lo conocemos) así que tendremos que restar 1 día a la fecha para la columna P48_CARBON, creando la columna adicional P48_CARBON_D-1

Aprovechamos esta operación para crear la columna RESULTADO D-1 que es la columna reserva del día anterior.

In [14]:
df_esios_ant= df_esios[['datetime','P48_CARBON','RESERVA']]

In [15]:
df_esios_ant.head()

,datetime,P48_CARBON,RESERVA
0,2015-10-01 00:00:00,8203.3,1.0
1,2015-10-01 01:00:00,8286.8,0.0
2,2015-10-01 02:00:00,8301.3,0.0
3,2015-10-01 03:00:00,8281.3,0.0
4,2015-10-01 04:00:00,8247.7,0.0


In [17]:
df_esios_ant.dtypes

datetime       object
P48_CARBON    float64
RESERVA       float64
dtype: object

In [18]:
dia=timedelta(days=1) 

In [47]:
fecha=pd.to_datetime(df_esios_ant['datetime'])
fecha_ant=pd.Series([x + dia for x in fecha])
fecha_ant.head()

0   2015-10-02 00:00:00
1   2015-10-02 01:00:00
2   2015-10-02 02:00:00
3   2015-10-02 03:00:00
4   2015-10-02 04:00:00
dtype: datetime64[ns]

In [50]:
fecha_ant.max()  #me sobra esta fecha

Timestamp('2018-10-01 23:00:00')

In [49]:
fecha_ant.min()

Timestamp('2015-10-02 00:00:00')

Necesito los datos del día 30/09/2015 para alimentar al 01/10/2015. Ejecuto la consulta a esios para la fecha 30/09/2015.

Notebook descarga_esios.ipynb

In [52]:
import requests
import json

In [58]:
headers_={'Accept': 'application/json; application/vnd.esios-api-v1+json',
          'Content-Type': 'application/json',
          'Host': 'api.esios.ree.es',
          'Authorization': 'Token token="9d6bcd627698602fbcd18721cca88b90d0f6e6025963f86be84fa18c87801a10"',
          'Cookie':''
         }

query_parametros_={'start_date': '2015-09-30 00:00:00',
                   'end_date':'2015-09-30 23:59:59'}

url_='https://api.esios.ree.es/indicators/' + str(10008)
    
#Hacemos peticion:
r = requests.get(url_,headers=headers_,params=query_parametros_)

data=json.loads(r.content)
    
df_data=pd.DataFrame(data['indicator']['values'])['value']
df_data


0     7181.4
1     6771.7
2     6559.5
3     6560.4
4     6724.1
5     7174.6
6     7955.3
7     8281.3
8     8304.3
9     8263.3
10    8063.3
11    8274.3
12    7935.3
13    8037.3
14    8278.3
15    8259.7
16    8083.3
17    8237.3
18    8297.3
19    8347.0
20    8299.6
21    8352.3
22    8300.3
23    8265.3
Name: value, dtype: float64